In [1]:
import hashlib
import json
from time import time

class Bloco:
    def __init__(self, index, dados, hash_anterior=''):
        self.index = index
        self.timestamp = time()
        self.dados = dados
        self.hash_anterior = hash_anterior
        self.nonce = 0
        self.hash_atual = self.gerar_hash()

    def gerar_hash(self):
        conteudo_bloco = json.dumps({
            'index': self.index,
            'timestamp': self.timestamp,
            'dados': self.dados,
            'hash_anterior': self.hash_anterior,
            'nonce': self.nonce
        }, sort_keys=True).encode()
        return hashlib.sha256(conteudo_bloco).hexdigest()

    def prova_de_trabalho(self, dificuldade):
        while self.hash_atual[:dificuldade] != '0' * dificuldade:
            self.nonce += 1
            self.hash_atual = self.gerar_hash()

class Blockchain:
    def __init__(self):
        self.cadeia = [self.criar_bloco_genesis()]
        self.dificuldade = 2

    def criar_bloco_genesis(self):
        return Bloco(0, "Bloco Genesis", "0")

    def obter_ultimo_bloco(self):
        return self.cadeia[-1]

    def adicionar_bloco(self, novo_bloco):
        novo_bloco.hash_anterior = self.obter_ultimo_bloco().hash_atual
        novo_bloco.prova_de_trabalho(self.dificuldade)
        self.cadeia.append(novo_bloco)

    def validar_cadeia(self):
        for i in range(1, len(self.cadeia)):
            bloco_atual = self.cadeia[i]
            bloco_anterior = self.cadeia[i-1]

            if bloco_atual.hash_atual != bloco_atual.gerar_hash():
                return False
            if bloco_atual.hash_anterior != bloco_anterior.hash_atual:
                return False
        return True

    def salvar_em_json(self, nome_arquivo='blockchain.json'):
        with open(nome_arquivo, 'w') as arquivo:
            json.dump([bloco.__dict__ for bloco in self.cadeia], arquivo, indent=4)

def coletar_dados_exame():
    glicose = float(input("Digite o nível de glicose: "))
    colesterol = float(input("Digite o nível de colesterol: "))
    return {
        'glicose': glicose,
        'colesterol': colesterol
    }


In [ ]:
blockchain = Blockchain()



In [ ]:
dados_exame = coletar_dados_exame()
novo_bloco = Bloco(blockchain.obter_ultimo_bloco().index + 1, dados_exame)
blockchain.adicionar_bloco(novo_bloco)

if blockchain.validar_cadeia():
    blockchain.salvar_em_json()
    print("Blockchain válida e salva!")
else:
    print("Erro na validação da blockchain!")


In [ ]:
def print_json_content(filename):

  try:
    with open(filename, 'r') as file:
      data = json.load(file)
      print(json.dumps(data, indent=4))  # Imprime o conteúdo com indentação para melhor legibilidade
  except FileNotFoundError:
    print(f"Erro: Arquivo '{filename}' não encontrado.")
  except json.JSONDecodeError:
    print(f"Erro: O arquivo '{filename}' não é um JSON válido.")

print_json_content('blockchain.json')